In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

Abrindo os dados e arrumando a coluna de metas, em seguida checando os datatypes

In [3]:
dados_final = 'orcamento_limpo_final_3.xlsx'
df = pd.read_excel(dados_final)

In [4]:
def arrumar_num_meta(item):
    
    item = str(item)

    antes, depois = item.split('.')
    
    if int(depois)>0:
        return f'{antes}.{depois}'
    else:
        return antes

In [5]:
df['meta'] = df['meta'].apply(arrumar_num_meta)

In [6]:
df.dtypes

Unnamed: 0                            int64
Objetivo Estratégico Atualizado      object
secretaria                           object
meta                                 object
IN                                   object
Item                                 object
Custeio/\nInvestimento               object
Código Ação                          object
Unidade Orçamentária                 object
Ação                                 object
Fonte                                object
classif.                             object
V. Atualizado                       float64
V. Congelado                        float64
V. Disponível                       float64
Atualizado (Disponibilizado PDM)    float64
Congelado (Dispon. PDM)             float64
Disponível (Dispon. PDM)            float64
Custo 2021                          float64
Custo 2022                          float64
Custo 2023                          float64
Custo 2024                          float64
Custo TOTAL                     

Veio uma coluna a mais que deve ser o index, vou deletar ela abaixo

In [7]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

Abaixo exploro a classificação das linhas. Há tanto itens de execução como contrapartida.

In [8]:
df['classif.'].unique()

array(['execução', 'contrapartida'], dtype=object)

In [9]:
linhas_contrapartidas = df['classif.']=='contrapartida'

In [10]:
contrapartidas = df[linhas_contrapartidas].copy()

In [11]:
contrapartidas['Custo TOTAL'].sum()

220345922.33071828

In [12]:
contrapartidas[contrapartidas['Custo TOTAL']>0]

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL
3,Garantir à população atendimento integral em s...,SMS,3.1,a,2,Investimento,84.11.9204.00,84.11,9204,00,contrapartida,80002000.0,2000.0,80000000.0,NaN,NaN,NaN,2.764035e+07,5.739074e+07,8321799.61,993968.93,9.434686e+07
497,Garantir à população atendimento integral em s...,SMS,3.2,A,2,investimento,84.11.9204.01,84.11,9204,01,contrapartida,NaN,0.0,223111204.0,NaN,NaN,NaN,8.241338e+07,3.785169e+07,1999785.13,0.00,1.222649e+08
499,Garantir à população atendimento integral em s...,SMS,3.2,B,2,investimento,84.11.9204.01,84.11,9204,01,contrapartida,223111204.0,0.0,223111204.0,NaN,NaN,NaN,2.238387e+06,1.495822e+06,0.00,0.00,3.734209e+06


Como há linhas de contrapartida que possuem valores para as colunas de custo, decidi removê-las do cálculo.

In [13]:
df = df[~linhas_contrapartidas].copy()

In [14]:
df['classif.'].unique()

array(['execução'], dtype=object)

Agora checamos para ver se há algum objetivo estratégico sem informação.

Como veremos, faltou informar para uma meta, o que faço manualmente abaixo.

In [15]:
df[df['Objetivo Estratégico Atualizado'].isnull()]

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL
5,NaN,SMS,5.1,c,1,Custeio,84.28.2509.00,84.28,2509,00,execução,5805541.0,1000.0,5804541.0,NaN,NaN,NaN,464000.00,144000.00,144000.00,144000.00,896000.00
6,NaN,SMS,5.1,c,2,Custeio,84.28.2509.00,84.28,2509,00,execução,5805541.0,1000.0,5804541.0,NaN,NaN,NaN,122815.00,38115.00,38115.00,38115.00,237160.00
7,NaN,SMS,5.1,c,3,Custeio,84.28.2509.00,84.28,2509,00,execução,5805541.0,1000.0,5804541.0,NaN,NaN,NaN,100796.40,NaN,NaN,NaN,100796.40
8,NaN,SMS,5.1,d,1,Custeio,84.22.2818.00,84.22,2818,00,execução,315750.0,0.0,315750.0,NaN,NaN,NaN,416520.00,138840.00,138840.00,138840.00,833040.00
9,NaN,SMS,5.1,d,2,Custeio,84.22.2818.00,84.22,2818,00,execução,315750.0,0.0,315750.0,NaN,NaN,NaN,23395.32,7798.44,7798.44,7798.44,46790.64


In [16]:
objetivo_meta_5_1 = ('Garantir a proteção integral e o pleno desenvolvimento para crianças de 0 a 6 anos de idade.')

df['Objetivo Estratégico Atualizado'].fillna(objetivo_meta_5_1,
                                            inplace = True)

In [17]:
df['Objetivo Estratégico Atualizado'].isnull().sum()

0

Abaixo checamos se há alguma linha sem informação da meta. Como vemos, não há

In [18]:
df['meta'].isnull().sum()

0

Agora vou identificar se o custo total está batendo

In [19]:
def somar_custos(row):
    
    anos = ['Custo '+ str(i) for i in range(2021, 2025)]
    
    soma = 0
    for col in anos:
        val = row[col]
        if pd.isnull(val):
            val = 0
        soma+=val
    
    return soma

In [20]:
df['soma_custo_total_H'] = df.apply(somar_custos, axis = 1)

In [21]:
df['dif_custo_total'] = df['Custo TOTAL'] - df['soma_custo_total_H']

In [22]:
tem_dif = df[df['dif_custo_total']>0].copy()

In [23]:
len(tem_dif)

18

In [24]:
tem_dif

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL,soma_custo_total_H,dif_custo_total
18,"Promover o acesso à moradia, à urbanização e à...",SEHAB,11,A/B,"1, 2 / 1,2",Investimento,XX..X.X.XX.XX,XX.,X,X,execução,NaN,NaN,NaN,2.311317e+08,1.305813e+08,1.005504e+08,2.326935e+08,1.104151e+09,9.868817e+08,8.804537e+08,3.204180e+09,3.204180e+09,4.768372e-07
72,Promover a inclusão e a melhoria na qualidade ...,SMPED,19,A,6,custeio,36.10.5407.00,36.10,5407,00,execução,2870000.0,0.0,2870000.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.115435e+04,4.057718e+04,4.463489e+04,1.663664e+05,1.663664e+05,2.910383e-11
129,Ampliar a capacidade de monitoramento em segur...,SMSU,27,C,2,Custeio,38.10.2192.00,38.10,2192,00,execução,30471664.0,3000.0,30468664.0,8.000000e+06,0.000000e+00,8.000000e+06,6.490131e+06,8.289150e+06,8.721825e+06,9.131728e+06,3.263283e+07,3.263283e+07,3.725290e-09
136,"Ampliar a resiliência da cidade às chuvas, red...",SIURB,30,E,1,Custeio,xx..x.x.xx.xx,xx.,x,x,execução,0.0,0.0,0.0,1.089553e+08,4.899380e+06,1.040559e+08,1.097515e+08,1.097515e+08,1.097515e+08,1.097515e+08,4.390062e+08,4.390062e+08,1.999998e-02
145,Garantir a qualidade e segurança das vias públ...,SMSUB,33,A,3,Custeio,12.10.2340.00,12.10,2340,00,execução,111335013.0,0.0,111335013.0,0.000000e+00,0.000000e+00,NaN,1.624554e+08,1.787009e+08,1.965710e+08,2.162281e+08,7.539555e+08,7.539555e+08,1.192093e-07
150,Garantir a qualidade e segurança das vias públ...,SIURB,35,C,3,Investimento,98.22.5287.08,98.22,5287,08,execução,8244700.0,8244700.0,0.0,8.244700e+06,8.244700e+06,0.000000e+00,4.430220e+05,3.248828e+05,3.174991e+05,3.101154e+05,1.395519e+06,1.395519e+06,2.328306e-10
250,Estimular a mobilidade ativa de maneira segura...,SMT; SMPED,39,c,22,Custeio,98.20.3757.08,98.20,3757,08,execução,18150000.0,18150000.0,0.0,4.485000e+04,4.485000e+04,0.000000e+00,4.485000e+04,4.485000e+04,9.420480e+05,9.420480e+05,1.973796e+06,1.973796e+06,1.000000e-02
261,Estimular a mobilidade ativa de maneira segura...,SMT,41,a,10,Investimento,98.20.1097.08,98.20,1097,08,execução,30546000.0,30546000.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.081678e+07,1.261958e+07,1.261958e+07,3.605595e+07,3.605595e+07,7.450581e-09
262,Estimular a mobilidade ativa de maneira segura...,SMT,41,a,11,Custeio,98.20.1097.08,98.20,1097,08,execução,30546000.0,30546000.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.384419e+06,1.615155e+06,1.615155e+06,4.614729e+06,4.614729e+06,1.000000e-02
264,Garantir o acesso ao Sistema Municipal de Tran...,SMT,42,d,1,Investimento,20.10.1095.00,20.10,1095,00,execução,1000.0,1000.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,6.000000e+06,0.000000e+00,6.000000e+06


Como vimos, há alguns casos em que o custo não bate, vou salvar eles abaixo para controle.

In [25]:
tem_dif.to_excel('items_orcamento_custo_total_nao_bate_2.xlsx')

Como pode ser uma questão metodológica das secretarias, continuar usando a coluna original.

Apenas vou preencher os valores nulos para não dar erro (com zero, no caso).

In [26]:
df['Custo TOTAL'].isnull().sum()

2

In [27]:
df['custo_total'] = df['Custo TOTAL'].fillna(0)
df['custo_total_2021'] = df['Custo 2021'].fillna(0)
df['custo_total_2022'] = df['Custo 2022'].fillna(0)
df['custo_total_2023'] = df['Custo 2023'].fillna(0)
df['custo_total_2024'] = df['Custo 2024'].fillna(0)

In [28]:
colunas_custos = [
    'custo_total_2021',
    'custo_total_2022',
    'custo_total_2023',
    'custo_total_2024',
    'custo_total'
]

Agora vou fazer os dois agrupamentos

In [29]:
por_objetivo = df.groupby('Objetivo Estratégico Atualizado').sum()[colunas_custos]

In [30]:
por_meta = df.groupby('meta').sum()[colunas_custos]

In [31]:
por_objetivo.head()

,custo_total_2021,custo_total_2022,custo_total_2023,custo_total_2024,custo_total
Objetivo Estratégico Atualizado,,,,,
Ampliar a capacidade de monitoramento em segurança urbana e a cobertura das ações protetivas destinadas às vítimas de violência na Cidade de São Paulo.,2.293049e+07,4.142283e+07,6.230660e+07,9.868308e+07,2.253430e+08
"Ampliar a resiliência da cidade às chuvas, reduzindo as áreas inundáveis e mitigando os prejuízos causados à população.",9.749078e+08,9.533722e+08,8.936658e+08,7.353704e+08,3.557316e+09
Ampliar o respeito à diversidade e fomentar a igualdade de oportunidades.,2.482576e+07,1.950442e+07,2.068723e+07,2.448204e+07,8.949946e+07
"Assegurar o acesso à internet como direito fundamental, promover a inclusão digital e a expansão da economia criativa na cidade.",6.007353e+05,9.007353e+05,9.007353e+05,6.007353e+05,1.015000e+08
"Atingir grau de excelência em segurança viária, com foco na diminuição do número de acidentes e de vítimas fatais no trânsito.",2.662948e+07,1.621010e+08,2.318092e+08,1.597421e+08,5.802817e+08


In [32]:
por_meta.head()

,custo_total_2021,custo_total_2022,custo_total_2023,custo_total_2024,custo_total
meta,,,,,
10,6.425672e+08,1.695373e+09,1.567173e+09,1.720243e+09,5.625356e+09
11,2.338135e+08,1.105271e+09,9.880017e+08,8.815737e+08,3.208660e+09
12,1.660148e+08,1.540131e+08,1.062047e+08,1.062047e+08,5.324373e+08
14,3.976540e+06,1.196540e+07,1.235140e+07,1.268980e+07,4.098314e+07
15,2.396800e+06,1.793500e+07,2.717760e+07,3.123840e+07,7.874780e+07


In [33]:
por_objetivo.to_excel('custos_totais_por_objetivo_3.xlsx')
por_meta.to_excel('custos_totais_por_meta_3.xlsx')